#### 사용자 정의 함수

****
- 함수 기능 : 모델 학습 진행 함수  
- 함수 이름 : training   
- 매개 변수 : 함수 구동 시 필요한 재료 >> 학습을 위한 재료   
            * 모델 인스턴스   
            * 학습 데이터셋 : feature, target(label) (학습전 Tensor화)   
            * 손실함수 인스턴스    
            * 최적화 인스턴스    
            * 학습 횟수 : 에포크(EPOCH)   
            * 배치 크기 : BATCH_SIZE    
            * 배치 개수 : BATCH_CNT    
            * 검증용 데이터셋 : feature, target (학습전 Tensor화)    
- 함수 결과 : 학습 시 에포크당 손실값과 성능지표값, 검증의 손실값과 성능지표값     
***

***
- 함수 기능 : 에포크 단위 모델 학습 진행 함수
- 함수 이름 : EpochTraining
- 매개 변수 : 함수 구동 시 필요한 재료 >> 학습을 위한 재료  
            * 모델 인스턴스   
            * 학습 데이터셋 : feature, target(label) (학습전 Tensor화)   
            * 손실함수 인스턴스   
            * 최적화 인스턴스    
            * 배치크기 : BATCH_SIZE    
            * 배치 개수 : BATCH_CNT     
- 함수 결과 : 손실값과 성능지표값   >> tuple로 담아서

***

In [1]:
# 모듈 로딩

import torch                        # 텐서 및 수치 계산 함수 관련 모듈
import torch.nn as nn               # 인공신경망 관련 모듈
import torch.nn.functional as F     # 손실, 거리 등 함수 관련 모듈
import torch.optim as optimizer     # 최적화 기법 관련 모듈
import pandas as pd                 # 데이터 파일 분석 관련 모듈
from sklearn.model_selection import train_test_split
from torchmetrics.regression import R2Score  # 성능 지표 관련 모듈 
from torchmetrics.classification import F1Score # 성능 지표 관련 모듈
from torchinfo import summary       # 모델 정보 관련 모듈

# 텐서 저장 및 실행 위치 설정
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
## 에포크 당 학습 진행 후 손실값과 성능지표값 반환 함수 >>> 여긴 뷴류 회귀 받는 함수 만들기
def EpochTraining(model, feature, target, lossFunc, optimizer, batch_cnt, batch_size = 32): # 디폴트값으로 주고싶은 것은 뒤로 보내고 값을 지정해주면 됩니다.
    # 에포크에서 배치크기만큼 데이터셋 추출 후 학습 진행
    loss_total, score_total = 0, 0
    for batch in range(batch_cnt):
        # 배치크기만큼 데이터셋 추출 및 덴서화
        start = batch * batch_size # 예) batch_size=32 >> 0,32,64,...
        end = start + batch_size # 예) batch_size=32 >> 32,64,96...

        X_train = torch.FloatTensor(feature[start:end].values).to(DEVICE)
        y_train = torch.FloatTensor(target[start:end].values).to(DEVICE)

        # 학습 진행
        pre_y = model(X_train)

        # 손실 계산 >> 변경
        loss = lossFunc(pre_y, y_train)
        loss_total += loss

        # 점수 추출   >> 변경                                #()까지 해야 인스턴스 생성
        score = F1Score()(pre_y, y_train) if is_class else R2Score()(pre_y, y_train) # 여기서 분류인지 회귀인지 분류
        score_total += score

        # 최적화 
        # - 분류 >>
        # - 회귀 >> 정답에 가까운 가중치(W)와 절편(b)을 찾음
            # 지금은 : W, b를 업데이트
        optimizer.zero_grad()
        loss.bachkward()
        optimizer.step()

    # 에포크당 손실값과 성능지표값 판정
    # 테스트 및 검증 함수 사용 >> 검증하는이유 >> 과대, 과소적합인지 아닌지, 학습이 멈출지 말지(오래하면 비례하다 떨어지는경우가 있기에 그시점을 알고 끊어 주기위해) 
    return loss_total/batch_cnt, 0

In [3]:
## 에포크 당 학습 진행 후 손실값과 성능지표값 반환 함수
def EpochTraining(model, feature, target, lossFunc, optimizer, batch_cnt, batch_size = 32, is_class=True): # 디폴트값으로 주고싶은 것은 뒤로 보내고 값을 지정해주면 됩니다.
    # 에포크에서 배치크기만큼 데이터셋 추출 후 학습 진행
    loss_total, score_total = 0, 0
    for batch in range(batch_cnt):
        # 배치크기만큼 데이터셋 추출 및 덴서화
        start = batch * batch_size # 예) batch_size=32 >> 0,32,64,...
        end = start + batch_size # 예) batch_size=32 >> 32,64,96...

        X_train = torch.FloatTensor(feature[start:end].values).to(DEVICE)
        y_train = torch.FloatTensor(target[start:end].values).to(DEVICE)

        # 학습 진행
        pre_y = model(X_train).to(DEVICE)

        # 손실 계산
        loss = lossFunc(pre_y, y_train)
        loss_total += loss

        # 점수 추출                                         #()까지 해야 인스턴스 생성
        score = F1Score()(pre_y, y_train) if is_class else R2Score()(pre_y, y_train) # 여기서 분류인지 회귀인지 분류
        score_total += score

        # 최적화 
        # - 분류 >> ?>????
        # - 회귀 >> 정답에 가까운 가중치(W)와 절편(b)을 찾음
            # 지금은 : W, b를 업데이트
        optimizer.zero_grad()
        loss.bachkward()
        optimizer.step()

    # 에포크당 손실값과 성능지표값 판정
    # 테스트 및 검증 함수 사용 >> 검증하는이유 >> 과대, 과소적합인지 아닌지, 학습이 멈출지 말지(오래하면 비례하다 떨어지는경우가 있기에 그시점을 알고 끊어 주기위해) 
    return loss_total/batch_cnt, 0

In [ ]:
## 검증 및 테스트 진행 후 손실값과 성능지표값 반환 함수
def testing(model, feature, target, lossFunc, scoreFunc):
    # 최적화 기능 비활성화 후 데이터셋 추출 후 학습 진행
    loss_total, score_total = 0, 0
    with  torch.no_grad():
        # 데이터셋 덴서ghk
        featureTS = torch.FloatTensor(feature.values).to(DEVICE)
        targetTS = torch.FloatTensor(target.values).to(DEVICE)

        # 학습 진행
        pre_y = model(featureTS).to(DEVICE)

        # 손실 계산
        loss = lossFunc(pre_y, targetTS)
        loss_total += loss

        # 점수 추출                       
        score = scoreFunc(pre_y, targetTS)
    
    # 손실값과 성능지표값 변환
    return loss, score


In [ ]:
## 학습 진행
EPOCHS =10
TV_LOSS = {'TRAIN':[], 'VAL':[]}
TV_SCORE = {'TRAIN':[], 'VAL':[]}

for epoch in range(EPOCHS):
    
    # 학습 진행
    train_loss, train_score = EpochTraining()
    
    # 검증 진행
    val_loss, val_score = testing()

    # 에포크당 학습 및 검증 결과 저장
    TV_LOSS['TRAIN'].append(train_loss)
    TV_SCORE['TRAIN'].append(train_score)
    
    TV_LOSS['Val'].append(val_loss)
    TV_SCORE['Val'].append(val_score)

    # 에포크당 학습 및 검증 결과 출력
    print(f'[{epoch}/{EPOCHS}]\n -[Train] LOSS : {train_loss}  SCORE : {train_score}')
    print(f'[{epoch}/{EPOCHS}]\n -[Val] LOSS : {val_loss}  SCORE : {val_score}')